In [50]:
# ImageNet-1k Evaluation

            # Top-1 Accuracy
            # Top-5 Accuracy
            # Precision
            # Recall
            # F1-Score
            # Mean Average Precision (mAP)

In [ ]:
import os

# Change the current directory to root directory
new_directory = "../"
os.chdir(new_directory)

# Verify the current directory has changed
updated_directory = os.getcwd()
print("Updated Directory:", updated_directory)

## Combined Compute metrics

In [ ]:
##### JSON format

json_result = {
    "topk_accuracy": {
        "top1_accuracy_raw" : None,
        "top5_accuracy_raw" : None,
        "n_samples": None
    },
    "precision_recall_f1_mAP": {
        "0": {
            "precision_raw": None,
            "recall_raw": None,
            "f1_raw": None,
            "mAP_raw": None,
            "n_samples": None
        }
    }
}

In [32]:
##### sum 2 jsons

import pandas as pd


json1 = {
    "topk_accuracy": {
        "top1_accuracy_raw": 10,
        "top5_accuracy_raw": 20,
        "n_samples": 30
    },
    "precision_recall_f1_mAP": {
        "0": {
            "precision_raw": 40,
            "recall_raw": 50,
            "f1_raw": 60,
            "mAP_raw": 70,
            "n_samples": 80
        },
        "1": {
            "precision_raw": 35,
            "recall_raw": 45,
            "f1_raw": 55,
            "mAP_raw": 65,
            "n_samples": 75
        }
    }
}

json2 = {
    "topk_accuracy": {
        "top1_accuracy_raw": 5,
        "top5_accuracy_raw": 15,
        "n_samples": 25
    },
    "precision_recall_f1_mAP": {
        "0": {
            "precision_raw": 35,
            "recall_raw": 45,
            "f1_raw": 55,
            "mAP_raw": 65,
            "n_samples": 75
        },
        "1": {
            "precision_raw": 35,
            "recall_raw": 45,
            "f1_raw": 55,
            "mAP_raw": 65,
            "n_samples": 75
        }
    }
}


# Flatten the JSON & Convert the flattened JSON to a Pandas DataFrame

flattened_json = pd.json_normalize(json1)
df1 = pd.DataFrame.from_dict(flattened_json)

flattened_json = pd.json_normalize(json2)
df2 = pd.DataFrame.from_dict(flattened_json)


# Sum the corresponding parameters using pandas

result = df1.add(df2, fill_value=0).to_dict()

result


{'topk_accuracy.top1_accuracy_raw': {0: 15},
 'topk_accuracy.top5_accuracy_raw': {0: 35},
 'topk_accuracy.n_samples': {0: 55},
 'precision_recall_f1_mAP.0.precision_raw': {0: 75},
 'precision_recall_f1_mAP.0.recall_raw': {0: 95},
 'precision_recall_f1_mAP.0.f1_raw': {0: 115},
 'precision_recall_f1_mAP.0.mAP_raw': {0: 135},
 'precision_recall_f1_mAP.0.n_samples': {0: 155},
 'precision_recall_f1_mAP.1.precision_raw': {0: 70},
 'precision_recall_f1_mAP.1.recall_raw': {0: 90},
 'precision_recall_f1_mAP.1.f1_raw': {0: 110},
 'precision_recall_f1_mAP.1.mAP_raw': {0: 130},
 'precision_recall_f1_mAP.1.n_samples': {0: 150}}

In [33]:
####### list of multiple jsons

flattened_json = pd.json_normalize([json1, json2])
df_all = pd.DataFrame.from_dict(flattened_json)
df_all.head()

,topk_accuracy.top1_accuracy_raw,topk_accuracy.top5_accuracy_raw,topk_accuracy.n_samples,precision_recall_f1_mAP.0.precision_raw,precision_recall_f1_mAP.0.recall_raw,precision_recall_f1_mAP.0.f1_raw,precision_recall_f1_mAP.0.mAP_raw,precision_recall_f1_mAP.0.n_samples,precision_recall_f1_mAP.1.precision_raw,precision_recall_f1_mAP.1.recall_raw,precision_recall_f1_mAP.1.f1_raw,precision_recall_f1_mAP.1.mAP_raw,precision_recall_f1_mAP.1.n_samples
0,10,20,30,40,50,60,70,80,35,45,55,65,75
1,5,15,25,35,45,55,65,75,35,45,55,65,75


In [38]:
###### sum alltogether

column_sums = df_all.sum()
print(type(column_sums))
column_sums

<class 'pandas.core.series.Series'>


topk_accuracy.top1_accuracy_raw             15
topk_accuracy.top5_accuracy_raw             35
topk_accuracy.n_samples                     55
precision_recall_f1_mAP.0.precision_raw     75
precision_recall_f1_mAP.0.recall_raw        95
precision_recall_f1_mAP.0.f1_raw           115
precision_recall_f1_mAP.0.mAP_raw          135
precision_recall_f1_mAP.0.n_samples        155
precision_recall_f1_mAP.1.precision_raw     70
precision_recall_f1_mAP.1.recall_raw        90
precision_recall_f1_mAP.1.f1_raw           110
precision_recall_f1_mAP.1.mAP_raw          130
precision_recall_f1_mAP.1.n_samples        150
dtype: int64

In [39]:
column_sums["topk_accuracy.n_samples"]

55

In [41]:
f1_score_class_0 = column_sums["precision_recall_f1_mAP.0.f1_raw"] / column_sums["precision_recall_f1_mAP.0.n_samples"]
f1_score_class_0

0.7419354838709677

In [ ]:
import evaluate
import datasets
import typing
import numpy as np
from sklearn.metrics import top_k_accuracy_score, precision_score, recall_score, f1_score, average_precision_score



def compute_metrics_imagenet1k(predictions, references):

    predictions = np.array(predictions)
    references = np.array(references)
    labels = list(range(1000))
    counts_each_label = [np.count_nonzero(references == label) for label in labels]


    # Top-1 accuracy
    k = 1
    topk_indices = np.argsort(-predictions, axis=1)[:, :k] # descending order
    top1_accuracy_raw = top_k_accuracy_score(references, topk_indices, k=k)*len(predictions)

    # Top-5 Accuracy
    k = 5
    topk_indices = np.argsort(-predictions, axis=1)[:, :k] # descending order
    top5_accuracy_raw = top_k_accuracy_score(references, topk_indices, k=k)*len(predictions)

    # Precision

    precision_result = precision_score(
                    y_pred = predictions.argmax(axis=1), 
                    y_true = references, 
                    labels=labels, 
                    pos_label=1, 
                    average=None, 
                    zero_division=1)
    
    # Recall

    recall_result = recall_score(
                    y_pred = predictions.argmax(axis=1), 
                    y_true = references, 
                    labels=labels, 
                    pos_label=1, 
                    average=None, 
                    zero_division=1)
    

    # F1-Score

    f1_result = f1_score(
                    y_pred = predictions.argmax(axis=1), 
                    y_true = references, 
                    labels=labels, 
                    pos_label=1, 
                    average=None, 
                    zero_division=1)
    

    # Mean Average Precision (mAP)

    mAP_result = average_precision_score(
                    y_pred = predictions.argmax(axis=1), 
                    y_score = references, 
                    labels=labels, 
                    pos_label=1, 
                    average=None, 
                    zero_division=1)


    # json initialize

    results = {
        "topk_accuracy": {
            "top1_accuracy_raw" : top1_accuracy_raw,
            "top5_accuracy_raw" : top5_accuracy_raw,
            "n_samples": len(predictions)
        },
        "precision_recall_f1_mAP": {}
        }
    

    for i in range(1000):

        n_samples_of_label_i = counts_each_label[i]

        labelwise_results = {
            "precision_raw": precision_result[i]*n_samples_of_label_i,
            "recall_raw": recall_result[i]*n_samples_of_label_i,
            "f1_raw": f1_result[i]*n_samples_of_label_i,
            "mAP_raw": mAP_result[i]*n_samples_of_label_i,
            "n_samples": n_samples_of_label_i
        }
        results["precision_recall_f1_mAP"][str(i)] = labelwise_results


    return results


In [19]:
####### JSON sum

import json

# Example JSON objects
json1 = {"value1": 10, "value2": 20}
json2 = {"value1": 30, "value2": 40}
json3 = {"value1": 50, "value2": 60}

# Extract the values from each JSON object
values = [json1["value1"], json1["value2"], json2["value1"], json2["value2"], json3["value1"], json3["value2"]]

# Calculate the sum of the values
sum_values = sum(values)

# Create a new JSON object with the summed values
sum_json = {"sum": sum_values}

# Print the result
print("Sum of JSON objects:", sum_json)


Sum of JSON objects: {'sum': 210}


In [42]:
####### Top-1 Accuracy

import numpy as np
import evaluate

accuracyk = evaluate.load("KevinSpaghetti/accuracyk")

predictions = np.array([
    [3],
    [4],
    [1],
])

references = np.array([3, 4, 0])

results = accuracyk.compute(predictions=predictions, references=references)
print(results)


/media/toma/2TB_30May2023/OBJECT_DETECTION_L/object_detection/.venv/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


{'accuracy': 0.6666666666666666}


In [17]:
import numpy as np
from sklearn.metrics import top_k_accuracy_score, precision_score

references = np.array([3, 4, 1])

predictions = np.array([
    [0.         , 0.30434783, 0.04347826, 0.13043478, 0.2173913 ],
    [0.         , 0.09090909, 0.40909091, 0.36363636, 0.18181818],
    [0.61538462, 0.30769231, 0.07692308, 0.07692308, 0.15384615]
])

top3_indices = np.argsort(-predictions, axis=1)[:, :3] # descending order

top2 = top_k_accuracy_score(references, top3_indices, k=2) #, labels = [0,1,2,3,4])
print(top2)

top2_raw = top2* len(predictions)
n_samples = len(predictions)
print(top2_raw)
print(n_samples)

0.6666666666666666
2.0
3


In [5]:
####### Top-5 Accuracy


import numpy as np
import evaluate

accuracyk = evaluate.load("KevinSpaghetti/accuracyk")

predictions = np.array([
    [0, 7, 1, 3, 5],
    [0, 2, 9, 8, 4],
    [8, 4, 1, 2, 3],
])
references = np.array([3, 4, 5])

results = accuracyk.compute(predictions=predictions, references=references)
print(results)

{'accuracy': 0.6666666666666666}


In [49]:
#### get top 3 (say top 5) from full row of 5 elements (say 1000)

import numpy as np

predictions = np.array([
    [0.         , 0.30434783, 0.04347826, 0.13043478, 0.2173913 ],
    [0.         , 0.09090909, 0.40909091, 0.36363636, 0.18181818],
    [0.61538462, 0.30769231, 0.07692308, 0.07692308, 0.15384615]
])

# Get the indices of the top 3 maximum values in each row
top3_indices = np.argsort(-predictions, axis=1)[:, :3] # descending order


print(top3_indices)


[[1 4 3]
 [2 3 4]
 [0 1 4]]


In [39]:
####### Precision

import numpy as np
import evaluate

metric = evaluate.load("precision")

predictions = np.array([
    [0, 7, 1, 3, 5],
    [0, 2, 9, 8, 4],
    [8, 4, 1, 1, 3],
])
references = np.array([3, 4, 0])

results = metric.compute(predictions=predictions.argmax(axis=1), references=references,
                         pos_label=1, average=None, zero_division=1, labels = [0,1,2,3,4])
print(results)

labels = [0, 1, 2, 3, 4]

counts = [np.count_nonzero(references == label) for label in labels]

print(counts)

for i in labels:
    print(f"label {i}:\tprecision_accumulate = {results['precision'][i]* counts[i]}\tcount_accumulate = {counts[i]}")

for i in range(predictions.shape[0]):
    results = metric.compute(predictions=predictions[i:i+1, :].argmax(axis=1), references=references[i:i+1],
                            pos_label=1, average="weighted", zero_division=0, labels = [0,1,2,3,4])
    print(results)

{'precision': array([1., 0., 0., 1., 1.])}
[1, 0, 0, 1, 1]
label 0:	precision_accumulate = 1.0	count_accumulate = 1
label 1:	precision_accumulate = 0.0	count_accumulate = 0
label 2:	precision_accumulate = 0.0	count_accumulate = 0
label 3:	precision_accumulate = 1.0	count_accumulate = 1
label 4:	precision_accumulate = 1.0	count_accumulate = 1
{'precision': 0.0}
{'precision': 0.0}
{'precision': 1.0}


In [40]:
### COUNT

import numpy as np
import evaluate

metric = evaluate.load("precision")

predictions = np.array([3, 2, 0, 0])
references = np.array([3, 4, 0, 0])
labels = [0, 1, 2, 3, 4]

results = metric.compute(predictions=predictions, references=references,
                         pos_label=1, average=None, zero_division=1, labels = [0,1,2,3,4])
print(results)

labels = [0, 1, 2, 3, 4]

counts = [np.count_nonzero(references == label) for label in labels]

print(counts)

for i in labels:
    print(f"label {i}:\tprecision_accumulate = {results['precision'][i]* counts[i]}\tcount_accumulate = {counts[i]}")




{'precision': array([1., 1., 0., 1., 1.])}
[2, 0, 0, 1, 1]
label 0:	precision_accumulate = 2.0	count_accumulate = 2
label 1:	precision_accumulate = 0.0	count_accumulate = 0
label 2:	precision_accumulate = 0.0	count_accumulate = 0
label 3:	precision_accumulate = 1.0	count_accumulate = 1
label 4:	precision_accumulate = 1.0	count_accumulate = 1


In [41]:

####### Recall


import numpy as np
import evaluate

metric = evaluate.load("recall")

predictions = np.array([
    [0, 7, 1, 3, 5],
    [0, 2, 9, 8, 4],
    [8, 4, 1, 1, 3],
])
references = np.array([3, 4, 0])

results = metric.compute(predictions=predictions.argmax(axis=1), references=references,
                         pos_label=1, average=None, zero_division=1, labels = [0,1,2,3,4])
print(results)

labels = [0, 1, 2, 3, 4]

counts = [np.count_nonzero(references == label) for label in labels]

print(counts)

for i in labels:
    print(f"label {i}:\trecall_accumulate = {results['recall'][i]* counts[i]}\tcount_accumulate = {counts[i]}")

for i in range(predictions.shape[0]):
    results = metric.compute(predictions=predictions[i:i+1, :].argmax(axis=1), references=references[i:i+1],
                            pos_label=1, average="weighted", zero_division=0, labels = [0,1,2,3,4])
    print(results)


{'recall': array([1., 1., 1., 0., 0.])}
[1, 0, 0, 1, 1]
label 0:	recall_accumulate = 1.0	count_accumulate = 1
label 1:	recall_accumulate = 0.0	count_accumulate = 0
label 2:	recall_accumulate = 0.0	count_accumulate = 0
label 3:	recall_accumulate = 0.0	count_accumulate = 1
label 4:	recall_accumulate = 0.0	count_accumulate = 1
{'recall': 0.0}
{'recall': 0.0}
{'recall': 1.0}


In [45]:

####### F1-Score

import numpy as np
import evaluate

metric = evaluate.load("f1")

predictions = np.array([
    [0, 7, 1, 3, 5],
    [0, 2, 9, 8, 4],
    [8, 4, 1, 1, 3],
])
references = np.array([3, 4, 0])

results = metric.compute(predictions=predictions.argmax(axis=1), references=references,
                         pos_label=1, average=None, labels = [0,1,2,3,4])
print(results)

labels = [0, 1, 2, 3, 4]

counts = [np.count_nonzero(references == label) for label in labels]

print(counts)

for i in labels:
    print(f"label {i}:\tf1_accumulate = {results['f1'][i]* counts[i]}\tcount_accumulate = {counts[i]}")

for i in range(predictions.shape[0]):
    results = metric.compute(predictions=predictions[i:i+1, :].argmax(axis=1), references=references[i:i+1],
                            pos_label=1, average=None, labels = [0,1,2,3,4])
    print(results)


{'f1': array([1., 0., 0., 0., 0.])}
[1, 0, 0, 1, 1]
label 0:	f1_accumulate = 1.0	count_accumulate = 1
label 1:	f1_accumulate = 0.0	count_accumulate = 0
label 2:	f1_accumulate = 0.0	count_accumulate = 0
label 3:	f1_accumulate = 0.0	count_accumulate = 1
label 4:	f1_accumulate = 0.0	count_accumulate = 1


/media/toma/2TB_30May2023/OBJECT_DETECTION_L/object_detection/.venv/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1609: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, "true nor predicted", "F-score is", len(true_sum))


{'f1': array([0., 0., 0., 0., 0.])}


/media/toma/2TB_30May2023/OBJECT_DETECTION_L/object_detection/.venv/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1609: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, "true nor predicted", "F-score is", len(true_sum))


{'f1': array([0., 0., 0., 0., 0.])}
{'f1': array([1., 0., 0., 0., 0.])}


/media/toma/2TB_30May2023/OBJECT_DETECTION_L/object_detection/.venv/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1609: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, "true nor predicted", "F-score is", len(true_sum))


In [55]:
from sklearn.metrics import average_precision_score
from transformers import EvalPrediction

# chanelcolgate/average_precision

def calculate_map(predictions, label_ids):

    # Get true labels
    labels = label_ids

    # Calculate average precision for each class
    average_precisions = []
    num_classes = predictions.shape[1]
    for class_idx in range(num_classes):

        print("class {}:".format(class_idx), average_precision_score(labels == class_idx, predictions[:, class_idx],
                                                                     average=None, pos_label=1))

        average_precisions.append(average_precision_score(labels == class_idx, predictions[:, class_idx]))
        # print(average_precisions)

    # Calculate mean average precision (mAP)
    map_score = sum(average_precisions) / num_classes

    return map_score


predictions = np.array([
    [9, 4, 7, 8, 2, 0, 6, 5, 3, 1],
    [9, 1, 7, 8, 0, 3, 2, 5, 6, 4],
    [7, 0, 4, 8, 1, 6, 5, 2, 9, 3],
    [0, 5, 7, 4, 9, 3, 2, 1, 8, 6],
    [5, 9, 8, 2, 0, 7, 6, 3, 1, 4],
]) # say, predictions in float

# after argmax axis=1, result: [0 0 8 4 1]

# references = np.array([3, 4, 0, 2, 1])

references = np.array([0, 0, 8, 4, 1]) #(here, predictions == references)

results = calculate_map(predictions=predictions, label_ids=references)
print(results)


class 0: 1.0
class 1: 1.0
class 2: -0.0
class 3: -0.0
class 4: 1.0
class 5: -0.0
class 6: -0.0
class 7: -0.0
class 8: 1.0
class 9: -0.0
0.4


/media/toma/2TB_30May2023/OBJECT_DETECTION_L/object_detection/.venv/lib/python3.9/site-packages/sklearn/metrics/_ranking.py:891: UserWarning: No positive class found in y_true, recall is set to one for all thresholds.
  warnings.warn(
/media/toma/2TB_30May2023/OBJECT_DETECTION_L/object_detection/.venv/lib/python3.9/site-packages/sklearn/metrics/_ranking.py:891: UserWarning: No positive class found in y_true, recall is set to one for all thresholds.
  warnings.warn(
/media/toma/2TB_30May2023/OBJECT_DETECTION_L/object_detection/.venv/lib/python3.9/site-packages/sklearn/metrics/_ranking.py:891: UserWarning: No positive class found in y_true, recall is set to one for all thresholds.
  warnings.warn(
/media/toma/2TB_30May2023/OBJECT_DETECTION_L/object_detection/.venv/lib/python3.9/site-packages/sklearn/metrics/_ranking.py:891: UserWarning: No positive class found in y_true, recall is set to one for all thresholds.
  warnings.warn(
/media/toma/2TB_30May2023/OBJECT_DETECTION_L/object_detectio

In [ ]:
##### writing compute metrics function










In [ ]:
##### sample evaluation script with focalnet tiny model online available